In [19]:
import copy
from datetime import datetime
import math
import random

import pandas as pd

In [20]:
table = {
    'jagiellonia': 60,
    'slask': 60,
    # 'legia': 56,
    # 'gornik': 53,
    # 'lech': 53,
    # 'rakow': 52,
    # 'pogon': 52
}

odds_df = [
    ['jagiellonia', 'warta', 1.38, 5, 7.2],
    # ['lech', 'korona', 1.82, 4.1, 3.75],
    # ['legia', 'zaglebie', 1.49, 4.4, 6.2],
    # ['pogon', 'gornik', 1.77, 4.1, 3.95],
    # ['puszcza', 'piast', 3.2, 3.3, 2.25],
    # ['radomiak', 'widzew', 2.33, 3.35, 3],
    ['rakow', 'slask', 2.38, 3.65, 2.7],
    # ['ruch', 'cracovia', 2.7, 3.5, 2.46],
    # ['lks', 'mielec', 2.75, 3.2, 2.6]
]

In [21]:
odds_df = pd.DataFrame(columns=['team_home', 'team_away', 'OddsH', 'OddsD', 'OddsA'], data=odds_df)

odds_df['BookieProbH'] = odds_df.apply(lambda row: 1/row['OddsH'] / (1/row['OddsH'] + 1/row['OddsD'] + 1/row['OddsA']), axis=1)
odds_df['BookieProbD'] = odds_df.apply(lambda row: 1/row['OddsD'] / (1/row['OddsH'] + 1/row['OddsD'] + 1/row['OddsA']), axis=1)
odds_df['BookieProbA'] = odds_df.apply(lambda row: 1/row['OddsA'] / (1/row['OddsH'] + 1/row['OddsD'] + 1/row['OddsA']), axis=1)

odds_df['BookieProbTotal'] = odds_df.apply(lambda row: row['BookieProbH'] + row['BookieProbD'] + row['BookieProbA'], axis=1)

odds_df.head()

,team_home,team_away,OddsH,OddsD,OddsA,BookieProbH,BookieProbD,BookieProbA,BookieProbTotal
0,jagiellonia,warta,1.38,5.00,7.2,0.681354,0.188054,0.130593,1.0
1,rakow,slask,2.38,3.65,2.7,0.394705,0.257369,0.347925,1.0


In [22]:
start = datetime.now()

slask_wins = 0
sims = 10000

for i in range(sims):
    tmp_table = copy.deepcopy(table)
    for index, row in odds_df.iterrows():
        random_number = random.random()
        if random_number < row['BookieProbH']:
            try:
                tmp_table[row['team_home']] += 3
            except KeyError:
                pass
        elif random_number < row['BookieProbH'] + row['BookieProbD']:
            try:
                tmp_table[row['team_home']] += 1
            except KeyError:
                pass
            try:
                tmp_table[row['team_away']] += 1
            except KeyError:
                pass
        else:
            try:
                tmp_table[row['team_away']] += 3
            except KeyError:
                pass
    slask_won = True
    for k, v in tmp_table.items():
        if k != 'slask' and tmp_table['slask'] <= v:
            slask_won = False
    if slask_won:
        slask_wins += 1

print(f'Time: {datetime.now()-start}')
print(f'{slask_wins/sims}')
print(f'Fair odds: {math.ceil(1/(slask_wins/sims) * 100) / 100}')

Time: 0:00:00.115960
0.128
Fair odds: 7.82
